# Task_03. Collect SARS-CoV-2 N protein & (optional) RNA-dependent RNA proteins

## Data files
<ul>
    <li> The genome of Covid-19: '../data/2019nCoV_genomes.2020_03_27.fa'
    <li> The protein of Covid-19: '../data/2019nCoV_proteins.2020_03_27.fa'
    <li> The genome of MERS: '../data/MERS_genomes.2020_02_03.fa'
    <li> The genome of SARS: '../data/SARS_genomes.2020_02_03.fa'
    <li> Reference file for Covid-19: GCF_009858895 files
</ul>
    
## Procedures
<ol>
<li> Read GCF_009858895.2 genomic info file.
<li> Make N-protein cDNA sequence and translate it.
<li> Solve below questions
<li> Extraction results of N-protein cDNA and N-protein sequences.
</ol>

## Questions
<ol>
    <li> Q1. Compare protein sequence from cDNA to GCF_009858895.2_ASM985889v3_protein.fna. According to this information, extract N proteins          cDNA/proteins from other SARS-CoV2 genomes (either 02_03 or 03_27).
    <li> Q2. Which pairs are most similar ones? Which pairs are most different ones? From this information, extract N proteins cDNA/proteins            from other SARS & MERS genomes.
    <li> Q3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
</ol>

In [15]:
# The function to read sequences from a FASTA file
def read_fasta(tmp):
    r = dict()
    fi = open(tmp, 'r')
    for line in fi:
        if line.startswith('>'):
            h_g = line.strip().lstrip('>')
            r[h_g] = ''
        else:
            r[h_g] += line.strip().replace(' ', '')
    fi.close()
    return r

In [16]:
# Read genomes and define gff file
SARS_CoV2_ref_genomes = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
genome_COVID_ref = read_fasta(SARS_CoV2_ref_genomes)
CoV2_ref_idx = '../data/GCF_009858895.2_ASM985889v3_genomic.gff'

In [17]:
# Get index
def get_idx(t, p):
    fi = open(t, 'r')
    for line in fi.readlines():
        if "gene" in line and "Name={}".format(p) in line:
            idx_s = int(line.split()[3])-1
            idx_e = int(line.split()[4])-1
            idx = (idx_s, idx_e)
    return idx
N_idx = get_idx(CoV2_ref_idx, 'N')

In [18]:
# Get the cDNA seq of N protein
for seq in genome_COVID_ref.values():
    ref_N_cDNA = seq[N_idx[0]:N_idx[1]+1]
ref_Np_cDNA = {'N_prot_cDNA' : [ref_N_cDNA]}
def w_seq(fi, gc):
    f = open(fi, 'w')
    for g in list(gc.keys()):
        if len(gc[g]) == 1:
            f.write(">{}\n".format(g))
            f.write("{}\n".format(gc[g][0]))
        else:
            for i in range(len(gc[g])):
                f.write(">{} #{}\n".format(g, i+1))
                f.write("\n{}".format(gc[g][i]))
    f.close()
w_seq("ref_Np_cDNA.fa", ref_Np_cDNA)

In [19]:
# Translate it to protein seq
'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ transeq --sequence ref_Np_cDNA.fa --outseq ref_Np_protein.fa
'''

'\n(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ transeq --sequence ref_Np_cDNA.fa --outseq ref_Np_protein.fa\n'

In [20]:
# HW 3-1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna.
N_ref_prot = 'ref_Np_protein.fa'
N_ref_tl = read_fasta(N_ref_prot)

fi_ref_prot = '../data/GCF_009858895.2_ASM985889v3_protein.faa'
ref_prot = read_fasta(fi_ref_prot)

# Define compare funtion
def compare(a, b):
    if a == b:
        print("They are same")
    else:
        print("They are different")
        
# Compare it to ref_prot sequence
Np_from_cDNA = N_ref_tl['N_prot_cDNA_1'][:-1]
Np_from_ref = ref_prot['YP_009724397.2 nucleocapsid phosphoprotein [Wuhan seafood market pneumonia virus]']

compare(Np_from_cDNA, Np_from_ref)

They are same


In [21]:
# HW 3-2-1 Which pairs are most similar ones? Which pairs are most different ones? From this information, extract N proteins cDNA from SARS-CoV2 genomes.
'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/data$ makeblastdb -dbtype nucl -in 2019nCoV_genomes.2020_03_27.fa -out 2019nCoV_genomes.2020_03_27.blastdb
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastn -db ../data/2019nCoV_genomes.2020_03_27.blastdb -query ref_Np_cDNA.fa -outfmt 7 -out cDNA_Np_03_27_nblast.nbl
'''

# Define the function of similarities of each pairs    
def simi(seq_fi, prot_name):
    fi = open(seq_fi, 'r')
    match_per = []
    for li in fi.readlines():
        if li.startswith(prot_name):
            match_per.append(float(li.split()[2]))
    if len(match_per) != 0:
        max_match = max(match_per)
        min_match = min(match_per)
    elif len(match_per) == 0:
        max_match = 0
        min_match = 0
    fi.close()
    
    fi = open(seq_fi, 'r')
    max_names = []
    min_names = []
    for li in fi.readlines():
        if li.startswith(prot_name):
            if li.split()[2] == str(max_match):
                max_names.append(li.split()[1])
            if li.split()[2] == str(min_match):
                min_names.append(li.split()[1])
    print(max_names, "are the most similar, and the match percentage is {}% \n".format(max_match))
    print(min_names, "are the most different, and the match percentage is {}% \n".format(min_match))
    fi.close()

# Print out similarities of each pairs
Np_nblast = 'cDNA_Np_03_27_nblast.nbl'
simi(Np_nblast, 'N_prot_cDNA')

# Define the function to extract the N protein cDNA seq
def extract_cDNA(result_cDNA, seq_fi, prot_name):
    fi = open(seq_fi, 'r')
    result_cDNA_write = {key[:8] : result_cDNA[key] for key in result_cDNA.keys()}
    result_cDNA_seq = {}
    for li in fi.readlines():
        if li.startswith(prot_name):
            key_c = str(li.split()[1])
            idx_s = int(li.split()[8])-1
            idx_e = int(li.split()[9])-1
            result_cDNA_seq[key_c] = result_cDNA_write[key_c][idx_s:idx_e+1]
    fi.close()
    return result_cDNA_seq

# Extract the N protein cDNA seq
nucl_seq = read_fasta('../data/2019nCoV_genomes.2020_03_27.fa')
#extract_cDNA(nucl_seq, Np_nblast, 'N_prot_cDNA')

#Professor, I don't know why max_names is empty. When I applied the funtion 'simi' to SARS genomes, I acquired the most similar list. 
#However, if I applied to COVID19, I cannot acquired the most similar list.

[] are the most similar, and the match percentage is 100.0% 

['MT233522', 'MT246470', 'MT246451'] are the most different, and the match percentage is 99.762% 



In [22]:
# HW 3-2-2 Which pairs are most similar ones? Which pairs are most different ones? From this information, extract N proteins cDNA/proteins from other SARS-CoV2 genomes.
'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/2019nCoV_genomes.2020_03_27.fa --outseq 2019nCoV_genomes.2020_03_27_AA.fa -frame 6
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in 2019nCoV_genomes.2020_03_27_AA.fa -out 2019nCoV.2020_03_27_AA
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastp -db 2019nCoV.2020_03_27_AA -query ref_Np_protein.fa -outfmt 7 -out AA_Np_03_27_tblast.tbl
'''

# Print out similarities of each pairs
Np_tblast = 'AA_Np_03_27_tblast.tbl'
simi(Np_tblast, 'N_prot_cDNA_1')

# Define the function to extract the N protein AA seq
def extract_prot(result_prot, seq_fi, prot_name):
    fi = open(seq_fi, 'r')
    result_prot_write = {key[:10] : result_prot[key] for key in result_prot.keys()}
    result_prot_seq = {}
    for li in fi.readlines():
        if li.startswith(prot_name):
            key_c = str(li.split()[1])
            idx_s = int(li.split()[8])-1
            idx_e = int(li.split()[9])-1
            result_prot_seq[key_c] = result_prot_write[key_c][idx_s:idx_e+1]
    fi.close()
    return result_prot_seq

# Extract the N protein cDNA seq
protAA_seq = read_fasta('2019nCoV_genomes.2020_03_27_AA.fa')
#extract_cDNA(protAA_seq, Np_tblast, 'N_prot_cDNA_1')

[] are the most similar, and the match percentage is 100.0% 

['MT233522_2', 'MT246470_2', 'MT246451_2'] are the most different, and the match percentage is 99.524% 



In [23]:
# HW 3-3-1 Which protein is most similar to SARS-CoV2 N protein? Which protein is most different to SARS N protein?
'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/data$ makeblastdb -dbtype nucl -in SARS_genomes.2020_02_03.fa -out SARS_genomes.2020_02_03.blastdb
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastn -db ../data/SARS_genomes.2020_02_03.blastdb -query ref_Np_cDNA.fa -outfmt 7 -out SARS_cDNA_Np_03_27_nblast.nbl
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/SARS_genomes.2020_02_03.fa --outseq SARS_genomes.2020_02_03_AA.fa -frame 6
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in SARS_genomes.2020_02_03_AA.fa -out SARS_genomes.2020_02_03_AA
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastp -db SARS_genomes.2020_02_03_AA -query ref_Np_protein.fa -outfmt 7 -out SARS_AA_Np_03_27_tblast.tbl
'''

# Print out similarities of each pairs in nblast
SARS_Np_nblast = 'SARS_cDNA_Np_03_27_nblast.nbl'
simi(SARS_Np_nblast, 'N_prot_cDNA')

# Print out similarities of each pairs in tblast
SARS_Np_tblast = 'SARS_AA_Np_03_27_tblast.tbl'
simi(SARS_Np_tblast, 'N_prot_cDNA_1')

# Define the function to extract the N protein cDNA and AA seq in SARS OR MERS
def extract_SM(result, seq_fi, prot_name):
    fi = open(seq_fi, 'r')
    result_write = {key[:12] : result[key] for key in result.keys()}
    result_seq = {}
    for li in fi.readlines():
        if li.startswith(prot_name):
            key_c = str(li.split()[1])
            idx_s = int(li.split()[8])-1
            idx_e = int(li.split()[9])-1
            result_seq[key_c] = result_write[key_c][idx_s:idx_e+1]
    fi.close()
    return result_seq

# Extract the N protein cDNA seq
SARS_nucl_seq = read_fasta('../data/SARS_genomes.2020_02_03.fa')
#extract_SM(SARS_nucl_seq, SARS_Np_nblast, 'N_prot_cDNA')

# Extract the N protein cDNA seq
SARS_protAA_seq = read_fasta('SARS_genomes.2020_02_03_AA.fa')
#extract_SM(SARS_protAA_seq, SARS_Np_tblast, 'N_prot_cDNA_1')

['ENA|DQ084200|DQ084200.1', 'ENA|DQ084199|DQ084199.1', 'ENA|DQ022305|DQ022305.2'] are the most similar, and the match percentage is 88.862% 

['ENA|AY686863|AY686863.1'] are the most different, and the match percentage is 88.217% 

['KU973692.1_3', 'DQ898174.1_1', 'FJ882949.1_1', 'FJ882934.1_3', 'FJ882937.1_2', 'FJ882935.1_1', 'FJ882933.1_2', 'FJ882947.1_3', 'FJ882946.1_1', 'FJ882927.1_2', 'FJ882932.1_3', 'FJ882939.1_3', 'FJ882936.1_3', 'FJ882938.1_2', 'FJ882941.1_1', 'FJ882929.1_1', 'FJ882944.1_3', 'FJ882940.1_3', 'FJ882954.1_2', 'FJ882960.1_3', 'FJ882955.1_3', 'FJ882931.1_3', 'FJ882928.1_2', 'FJ882926.1_2', 'FJ882930.1_2', 'AY362698.1_1', 'AY394850.2_2'] are the most similar, and the match percentage is 90.544% 

['DQ084200.1_3', 'DQ022305.2_2'] are the most different, and the match percentage is 89.573% 



In [24]:
# HW 3-3-2 Which protein is most similar to SARS-CoV2 N protein? Which protein is most different to SARS N protein?
'''
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/data$ makeblastdb -dbtype nucl -in MERS_genomes.2020_02_03.fa -out MERS_genomes.2020_02_03.blastdb
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastn -db ../data/MERS_genomes.2020_02_03.blastdb -query ref_Np_cDNA.fa -outfmt 7 -out MERS_cDNA_Np_03_27_nblast.nbl
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/MERS_genomes.2020_02_03.fa --outseq MERS_genomes.2020_02_03_AA.fa -frame 6
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in MERS_genomes.2020_02_03_AA.fa -out MERS_genomes.2020_02_03_AA
(base) mirna@DESKTOP-BN3PD9C:~/git/BME203-2020/03_Annotation$ blastp -db MERS_genomes.2020_02_03_AA -query ref_Np_protein.fa -outfmt 7 -out MERS_AA_Np_03_27_tblast.tbl
'''

# Print out similarities of each pairs in nblast
MERS_Np_nblast = 'MERS_cDNA_Np_03_27_nblast.nbl'
simi(MERS_Np_nblast, 'N_prot_cDNA')

# Print out similarities of each pairs in tblast
MERS_Np_tblast = 'MERS_AA_Np_03_27_tblast.tbl'
simi(MERS_Np_tblast, 'N_prot_cDNA_1')

# Extract the N protein cDNA seq
MERS_nucl_seq = read_fasta('../data/MERS_genomes.2020_02_03.fa')
#extract_SM(MERS_nucl_seq, MERS_Np_nblast, 'N_prot_cDNA')

# Extract the N protein cDNA seq
MERS_protAA_seq = read_fasta('MERS_genomes.2020_02_03_AA.fa')
#extract_SM(MERS_protAA_seq, MERS_Np_tblast, 'N_prot_cDNA_1')

[] are the most similar, and the match percentage is 0% 

[] are the most different, and the match percentage is 0% 

['KX574227.1_3'] are the most similar, and the match percentage is 50.142% 

['KX108941.1_2'] are the most different, and the match percentage is 43.333% 

